In [ ]:
import numpy as np
import pandas as pd


data = pd.read_csv("../data/df_2_text_processed.csv")

data['total_sales_before_tax'] = data.groupby('order_id')['sales_before_tax'].transform('sum')
data['total_sales_inc_tax'] = data.groupby('order_id')['sales_inc_tax'].transform('sum')

data[~(data.guest_count == 0)]